## Feature Engineering

In [1]:
import pandas as pd
from dateutil import relativedelta, parser
from pandas.tseries.offsets import BDay
from datetime import datetime, date, time
import numpy as np
import pickle
import time
import re
import pandas_datareader.data as reader

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
df = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/wholething.pkl")

In [24]:
ft = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/wamzbfqpfebjta3d.csv")
ft = ft.drop(['SHRCLS', 'SHRFLG', 'FACPR', 'NMSIND', 'VOL', 'FACSHR'], axis=1)
ft = ft[ft["TICKER"].isin(df["symbol"])]
ft.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,PERMNO,date,SHRCD,EXCHCD,TICKER,NAICS,PRC,SHROUT,CFACPR,CFACSHR
0,10026,2018/01/02,11.0,3.0,JJSF,311821.0,149.17999,18668.0,1.0,1.0
1,10026,2018/01/03,11.0,3.0,JJSF,311821.0,147.69000,18668.0,1.0,1.0
2,10026,2018/01/04,11.0,3.0,JJSF,311821.0,149.73000,18668.0,1.0,1.0
3,10026,2018/01/05,11.0,3.0,JJSF,311821.0,148.30000,18668.0,1.0,1.0
4,10026,2018/01/08,11.0,3.0,JJSF,311821.0,148.41000,18668.0,1.0,1.0


In [25]:
# calculate market capitalization
ft["Mkt_Size"] = ft["PRC"]*ft["SHROUT"]
ft.head()

,PERMNO,date,SHRCD,EXCHCD,TICKER,NAICS,PRC,SHROUT,CFACPR,CFACSHR,Mkt_Size
0,10026,2018/01/02,11.0,3.0,JJSF,311821.0,149.17999,18668.0,1.0,1.0,2.784892e+06
1,10026,2018/01/03,11.0,3.0,JJSF,311821.0,147.69000,18668.0,1.0,1.0,2.757077e+06
2,10026,2018/01/04,11.0,3.0,JJSF,311821.0,149.73000,18668.0,1.0,1.0,2.795160e+06
3,10026,2018/01/05,11.0,3.0,JJSF,311821.0,148.30000,18668.0,1.0,1.0,2.768464e+06
4,10026,2018/01/08,11.0,3.0,JJSF,311821.0,148.41000,18668.0,1.0,1.0,2.770518e+06


In [ ]:
# define the function to get industries

def get_industry(row):
    tckr = row["symbol"]
    try:
      industry = ft[ft["TICKER"] == tckr]["NAICS"].values[0]
    except:
      industry = np.nan 
    return industry

In [ ]:
# apply the function
df["ind"] = df.apply(get_industry, axis = 1)

In [ ]:
#save to pickle
df.to_pickle("/content/drive/MyDrive/Colab Notebooks/wholething_ind.pkl")

In [26]:
df = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/wholething_ind.pkl")
df.head()

,symbol,cik,form,access_number,filed_date,accepted_date,report_url,filing_url,string,texts,lens,pct_change1,pct_change2,beta1,beta2,mkt_excess1,mkt_excess2,ind
0,NICK,1000045,8-K,0001193125-19-024617,2019-02-01,2019-02-01 06:31:07,https://www.sec.gov/Archives/edgar/data/100004...,https://www.sec.gov/Archives/edgar/data/100004...,8-K 1 d675768d8k.htm FORM 8-K\n UNITED STAT...,Item 2.02\nResults of Operations and Financial...,182,0.039772,0.018824,0.1235,0.12100,0.039502,0.017531,523910.0
1,MFIN,1000209,8-K,0001193125-19-004285,2019-01-08,2019-01-08 09:10:35,https://www.sec.gov/Archives/edgar/data/100020...,https://www.sec.gov/Archives/edgar/data/100020...,8-K 1 d685338d8k.htm 8-K\n UNITED STATES\nS...,Item 2.02 Results of Operations and Financial ...,711,0.005894,0.019647,0.9098,0.91085,-0.003435,0.005013,525990.0
2,MFIN,1000209,8-K,0001193125-19-007413,2019-01-11,2019-01-11 16:32:03,https://www.sec.gov/Archives/edgar/data/100020...,https://www.sec.gov/Archives/edgar/data/100020...,8-K 1 d682501d8k.htm FORM 8-K\n UNITED STAT...,ITEM 1.01.\nENTRY INTO A MATERIAL DEFINITIVE A...,261,-0.019268,-0.019268,0.8742,0.87420,-0.013888,-0.013888,525990.0
3,MFIN,1000209,8-K,0001193125-19-024926,2019-02-01,2019-02-01 08:50:35,https://www.sec.gov/Archives/edgar/data/100020...,https://www.sec.gov/Archives/edgar/data/100020...,8-K 1 d671008d8k.htm FORM 8-K\n UNITED STAT...,ITEM 1.01.\nENTRY INTO A MATERIAL DEFINITIVE A...,257,0.077505,0.035918,0.8901,0.88485,0.076161,0.027620,525990.0
4,MFIN,1000209,8-K,0001193125-19-047009,2019-02-21,2019-02-21 16:13:02,https://www.sec.gov/Archives/edgar/data/100020...,https://www.sec.gov/Archives/edgar/data/100020...,8-K 1 d711389d8k.htm 8-K\n UNITED STATES\nS...,ITEM 1.01.\nENTRY INTO A MATERIAL DEFINITIVE A...,257,0.076013,0.076013,0.9225,0.92250,0.070040,0.070040,525990.0


In [27]:
#check the number of unique sub-sectors and their counts
df.ind.value_counts()

325412.0    1632
522110.0    1551
525990.0     567
531110.0     528
511210.0     437
            ... 
812921.0       2
322220.0       2
112310.0       2
561611.0       1
711410.0       1
Name: ind, Length: 581, dtype: int64

In [28]:
df.ind.isnull().sum()

917

In [29]:
# replace null values with nan string
df.loc[df["ind"].isnull(), "ind"] = "nan"

In [30]:
# turn the industry type to string to get the first two digits/characters off all observations
df["ind"] = df["ind"].astype("string")

In [31]:
# get parent sectors
df["ind"] = df["ind"].str[:2]

# parent sector value counts
df.ind.value_counts()

52    4454
33    3688
32    3006
54    1535
51    1495
53    1236
21    1206
na     917
42     739
22     691
44     666
48     582
56     577
62     511
31     500
55     408
72     392
45     334
23     258
71     144
81      76
11      69
61      63
49      37
Name: ind, dtype: Int64

In [32]:
# check value counts of share types
ft.SHRCD.value_counts()

11.0    1626712
12.0      88689
18.0      84737
71.0      47219
14.0      24119
48.0      16723
73.0      16399
31.0       6446
74.0       4527
44.0       2012
41.0       1006
21.0        503
72.0        503
Name: SHRCD, dtype: int64

In [33]:
# check value counts of exhchange traded
ft.EXCHCD.value_counts()

 3.0    1046678
 1.0     770626
 2.0      80654
 4.0      20926
 5.0        325
 6.0        251
-1.0        135
Name: EXCHCD, dtype: int64

In [34]:
# get item numbers

pattern = r"Item \d{1,2}\.\d{2}"

def get_items(row):
    item_list = sorted(set(re.findall(pattern, row.lower(), re.IGNORECASE)))
    return item_list

df["items"] = df.texts.apply(get_items)

In [39]:
# get CBOE Volatility Index

start = datetime(2019, 1, 1)
end = datetime(2019, 8, 1)

vix = reader.DataReader("VIXCLS", "fred", start, end)
print(len(vix))
print(vix.tail())
print('\n')

153
            VIXCLS
DATE              
2019-07-26   12.16
2019-07-29   12.83
2019-07-30   13.94
2019-07-31   16.12
2019-08-01   17.87




In [ ]:
# get VIX data & add to df as column
def get_vix(row):
  h = int(row["accepted_date"].split()[1].split(":")[0])
    
  if h>=16:
    start_date =datetime.strptime(row["accepted_date"].split()[0], "%Y-%m-%d")
    start_date_str = start_date.strftime ("%Y-%m-%d")
    end_date = start_date + BDay(1)
    end_date_str = end_date.strftime ("%Y-%m-%d")
        
    v = vix[vix.index == end_date_str].values[0]   
    
  else:
    start_date =datetime.strptime(row["accepted_date"].split()[0], "%Y-%m-%d")
    start_date_str = start_date.strftime ("%Y-%m-%d")
    
    v = vix[vix.index == start_date_str].values[0]
  return v

In [ ]:
df["VIX"] = df.apply(get_vix, axis = 1)

In [ ]:
#get share type & add to df as column
def get_shrcd(row):
  tckr = row["symbol"]
  try:
    shrcd = ft[ft["TICKER"] == tckr]["SHRCD"].values[0]
  except:
    shrcd = np.nan
  return shrcd

In [ ]:
df["SHRCD"] = df.apply(get_shrcd, axis = 1)

In [ ]:
# get exchange code and & add to df as column
def get_exchange(row):
  tckr = row["symbol"]
  try:
    exchange = ft[ft["TICKER"] == tckr]["EXCHCD"].values[0]
  except:
    exchange = np.nan
  return exchange

In [ ]:
df["EXCHCD"] = df.apply(get_exchange, axis = 1)

In [ ]:
# get market capitalization & add to df as column
def get_size(row):
  tckr = row["symbol"]
  date = datetime.strptime(row["accepted_date"].split()[0], "%Y-%m-%d")
  date_str = date.strftime ("%Y/%m/%d")
  
  temp = ft[ft["TICKER"] == tckr]
  try:
    cap = temp[temp["date"] == date_str]["Mkt_Size"].values[0]
  except:
    cap = np.nan
  return cap

In [ ]:
df["Mkt_Cap"] = df.apply(get_size, axis=1)

In [ ]:
# get dummies of categorical variables
df = pd.get_dummies(data = df, columns = ["SHRCD", "ind", "EXCHCD", "items"], drop_first=True)

In [ ]:
# change VIX data to float
df["VIX"] = df["VIX"].astype("float")

In [ ]:
df.to_pickle("/content/drive/MyDrive/Colab Notebooks/with_all_ft.pkl")

In [46]:
dfr = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/with_all_ft.pkl")
dfr.dropna(inplace=True)
df = dfr.reset_index(drop = True)
df = df.drop(['symbol', 'cik', 'form', 'access_number', 'filed_date','accepted_date', 'report_url', 'filing_url', 'string', 'lens' ], axis=1)
df.head()

,texts,pct_change1,pct_change2,beta1,beta2,mkt_excess1,mkt_excess2,item 1.01,item 1.02,item 1.03,item 1.04,item 2.01,item 2.02,item 2.03,item 2.04,item 2.05,item 2.06,item 2.10,item 3.01,item 3.02,item 3.03,item 4.01,item 4.02,item 4.04,item 5.01,item 5.02,item 5.03,item 5.04,item 5.05,item 5.07,item 5.08,item 7.01,item 7.02,item 8.01,item 9.01,item 9.02,VIX,label1,label2,SHRCD_12.0,SHRCD_14.0,SHRCD_18.0,SHRCD_21.0,SHRCD_31.0,SHRCD_41.0,SHRCD_44.0,SHRCD_48.0,SHRCD_71.0,SHRCD_72.0,SHRCD_73.0,SHRCD_74.0,ind_21,ind_22,ind_23,ind_31,ind_32,ind_33,ind_42,ind_44,ind_45,ind_48,ind_49,ind_51,ind_52,ind_53,ind_54,ind_55,ind_56,ind_61,ind_62,ind_71,ind_72,ind_81,ind_na,EXCHCD_1.0,EXCHCD_2.0,EXCHCD_3.0,EXCHCD_4.0,Mkt_Cap
0,Item 2.02\nResults of Operations and Financial...,0.039772,0.018824,0.1235,0.12100,0.039502,0.017531,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,16.14,pos,pos,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,86355.36
1,Item 2.02 Results of Operations and Financial ...,0.005894,0.019647,0.9098,0.91085,-0.003435,0.005013,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,20.47,pos,pos,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,125102.08
2,ITEM 1.01.\nENTRY INTO A MATERIAL DEFINITIVE A...,-0.019268,-0.019268,0.8742,0.87420,-0.013888,-0.013888,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,19.07,neg,neg,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,126812.46
3,ITEM 1.01.\nENTRY INTO A MATERIAL DEFINITIVE A...,0.077505,0.035918,0.8901,0.88485,0.076161,0.027620,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,16.14,pos,pos,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,139273.80
4,ITEM 1.01.\nENTRY INTO A MATERIAL DEFINITIVE A...,0.076013,0.076013,0.9225,0.92250,0.070040,0.070040,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,13.51,pos,pos,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,144649.28
